In [1]:
import numpy as np
import copy

## Reward definition

In [13]:
reward = np.zeros((3, 3, 2))
reward[0, 1, 0] = 1.0
reward[0, 2, 0] = 2.0
reward[0, 0, 1] = 0.0
reward[1, 0, 0] = 1.0
reward[1, 2, 0] = 2.0
reward[1, 1, 1] = 1.0
reward[2, 0, 0] = 1.0
reward[2, 1, 0] = 0.0
reward[2, 2, 1] = -1.0

### MDP setting
* state: index of each arrays
  * 0: Home, 1: Office, 2: Bar
* MDP array describes the probabilities for move to next state if choiced moving

In [3]:
transition_prob = [0.8, 0.5, 1.0]

In [4]:
def get_next_state(current, next_prob_arr, unif):
    next_prob = next_prob_arr[current]
    if unif <= next_prob:
        return (current + 1) % 3
    else:
        return (current + 2) % 3

In [5]:
get_next_state(0, transition_prob, 0.5)

1

* move: 0, stay: 1

In [11]:
move = 0
stay = 1
def policy(state, p, unif):
    move_prob = p[state]
    if unif <= move_prob:
        return move
    else:
        return stay

In [7]:
pi = [0.5, 0.5, 0.5]

In [8]:
init_state = 0

In [16]:
s = init_state
a = 0
r = 0.0
state_hist = []
action_hist = []
reward_hist = []

for i in range(0,10):
    unif = np.random.uniform()
    a = policy(s, pi, unif)
    
    if a == move:
        unif = np.random.uniform()
        next_s = get_next_state(s, transition_prob, unif)
    else:
        next_s = s
    
    r = reward[s, next_s, a]
    
    state_hist.append(s)
    action_hist.append(a)
    reward_hist.append(r)
    
    print('current state:', s, ' action:', a, ' next state:', next_s, ' reward:', r)
    
    s = next_s

current state: 0  action: 0  next state: 1  reward: 1.0
current state: 1  action: 1  next state: 1  reward: 1.0
current state: 1  action: 0  next state: 2  reward: 2.0
current state: 2  action: 0  next state: 0  reward: 1.0
current state: 0  action: 1  next state: 0  reward: 0.0
current state: 0  action: 1  next state: 0  reward: 0.0
current state: 0  action: 0  next state: 1  reward: 1.0
current state: 1  action: 0  next state: 0  reward: 1.0
current state: 0  action: 1  next state: 0  reward: 0.0
current state: 0  action: 1  next state: 0  reward: 0.0


In [14]:
state_hist

[0, 1, 1, 0, 1, 2, 2, 2, 2, 2]

In [15]:
reward_hist

[1.0, 1.0, 1.0, 1.0, 2.0, -1.0, -1.0, -1.0, -1.0, -1.0]

In [19]:
def monte_carlo_by_episode(state_hist, reward_hist):
    return 0